<a href="https://colab.research.google.com/github/jsansao/AprendizadoReforco/blob/main/Licao1_RL_Q_Learning_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Aprendizado por Reforço: Q-Learning com FrozenLake

Este é um notebook para demonstrar o **Q-Learning Tabular**, um dos algoritmos fundamentais de Aprendizado por Reforço.

Vamos treinar um agente para navegar no ambiente "FrozenLake" (Lago Congelado) da biblioteca `Gymnasium` (a sucessora do `Gym`).

**O Objetivo:**
O agente começa no estado 'S' (Start) e precisa chegar ao estado 'G' (Goal/Frisbee), evitando os 'H' (Holes/Buracos).

```
S F F F
F H F H
F F F H
H F F G
```

* **S**: Start (Início)
* **F**: Frozen (Congelado - seguro)
* **H**: Hole (Buraco - fim de jogo)
* **G**: Goal (Objetivo - vitória)


## Etapa 1: Instalar e Importar as Bibliotecas

Precisamos de:
* `gymnasium`: Para o ambiente (o jogo).
* `numpy`: Para criar e gerenciar nossa Q-Table.
* `random`: Para a exploração.

In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
import numpy as np
import random
import time

## Etapa 2: Configurar o Ambiente

Vamos carregar o ambiente. Usaremos `is_slippery=False` para tornar o problema determinístico por enquanto (se você mandar ir para 'direita', ele vai para 'direita'). Isso torna o aprendizado mais fácil de visualizar.

O modo `render_mode='ansi'` nos permite imprimir o jogo no console.

In [ ]:
env = gym.make('FrozenLake-v1', is_slippery=False, render_mode='ansi')

# Resetar o ambiente para o estado inicial
estado_inicial, info = env.reset()

# Mostrar o estado inicial
print("---- O Ambiente Inicial ----")
print(env.render())

# Espaços de Ação e Estado
num_estados = env.observation_space.n
num_acoes = env.action_space.n

print(f"Número de estados: {num_estados}")
print(f"Número de ações: {num_acoes}")
print("Ações: 0=Esquerda, 1=Baixo, 2=Direita, 3=Cima")

---- O Ambiente Inicial ----

SFFF
FHFH
FFFH
HFFG

Número de estados: 16
Número de ações: 4
Ações: 0=Esquerda, 1=Baixo, 2=Direita, 3=Cima


## Etapa 3: Inicializar a Q-Table

Aqui está o "cérebro" do nosso agente. É uma tabela (matriz) com `(número de estados) x (número de ações)`.

Vamos inicializar todos os valores Q(s, a) como zero.

In [ ]:
q_table = np.zeros((num_estados, num_acoes))

print("---- Q-Table Inicial (Tudo Zero) ----")
print(q_table)

---- Q-Table Inicial (Tudo Zero) ----
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Etapa 4: Definir os Hiperparâmetros

Estes são os "knobs" que ajustamos para o treinamento:

In [ ]:
# Taxa de aprendizado (Alpha)
# Quão rápido o agente aprende com novas informações.
alpha = 0.1

# Fator de desconto (Gamma)
# O quão importantes são as recompensas futuras (0 = só importa o agora, 1 = futuro é tão bom quanto).
gamma = 0.99

# Taxa de exploração (Epsilon)
# Chance de o agente tomar uma ação aleatória (explorar) vs. a melhor ação que ele conhece (explorar).
epsilon_inicial = 1.0       # Começa 100% aleatório
epsilon_min = 0.01          # Mínimo de 1% aleatório
taxa_decaimento_epsilon = 0.001 # Taxa que o epsilon diminui
epsilon = epsilon_inicial

# Número total de episódios (jogos) que vamos jogar para treinar
num_episodios = 20000

## Etapa 5: O Loop de Treinamento (Q-Learning)

Aqui está o coração do algoritmo, que segue a **Equação de Bellman**:

`Q(s, a) = Q(s, a) + α * [R + γ * max(Q(s', a')) - Q(s, a)]`

Em português:
`Novo_Valor_Q = Valor_Antigo + Taxa_Aprendizado * [Recompensa + Desconto * Valor_Máximo_Futuro - Valor_Antigo]`

In [ ]:
print("Iniciando o treinamento...")

for episodio in range(num_episodios):
    # 1. Resetar o ambiente para um novo jogo
    estado, info = env.reset()
    terminado = False
    truncado = False

    while not terminado and not truncado:
        # 2. Decidir Ação (Exploração vs. Explotação)
        if random.uniform(0, 1) < epsilon:
            acao = env.action_space.sample()  # Ação aleatória (Exploração)
        else:
            acao = np.argmax(q_table[estado, :]) # Melhor ação conhecida (Explotação)

        # 3. Tomar a Ação e observar o resultado
        novo_estado, recompensa, terminado, truncado, info = env.step(acao)

        # 4. Atualizar a Q-Table (Equação de Bellman)
        valor_antigo = q_table[estado, acao]
        valor_max_futuro = np.max(q_table[novo_estado, :])

        # A fórmula principal do Q-Learning
        novo_valor_q = valor_antigo + alpha * (recompensa + gamma * valor_max_futuro - valor_antigo)

        q_table[estado, acao] = novo_valor_q

        # 5. Atualizar o estado
        estado = novo_estado

    # Atualizar o Epsilon (Decaimento Exponencial)
    # O agente explora menos à medida que aprende mais
    epsilon = epsilon_min + (epsilon_inicial - epsilon_min) * np.exp(-taxa_decaimento_epsilon * episodio)

    if (episodio + 1) % 5000 == 0:
        print(f"Episódio {episodio + 1}/{num_episodios} concluído. Epsilon: {epsilon:.4f}")

print("Treinamento Concluído!")

Iniciando o treinamento...
Episódio 5000/20000 concluído. Epsilon: 0.0167
Episódio 10000/20000 concluído. Epsilon: 0.0100
Episódio 15000/20000 concluído. Epsilon: 0.0100
Episódio 20000/20000 concluído. Epsilon: 0.0100
Treinamento Concluído!


## Etapa 6: Ver a Q-Table Treinada

Vamos ver o que o agente aprendeu. Cada linha é um estado (0-15), e cada coluna uma ação (E, B, D, C).

Os valores mais altos indicam a ação "preferida" do agente em cada estado.

In [ ]:
print("---- Q-Table Final ----")
print(q_table)

---- Q-Table Final ----
[[0.94148015 0.95099005 0.93206535 0.94148015]
 [0.94148015 0.         0.79365997 0.86000295]
 [0.90193369 0.12759019 0.15175136 0.2207541 ]
 [0.44413131 0.         0.00396355 0.03690548]
 [0.95099005 0.96059601 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.88202222 0.         0.25169222]
 [0.         0.         0.         0.        ]
 [0.96059601 0.         0.970299   0.95099005]
 [0.960596   0.9801     0.9801     0.        ]
 [0.94687332 0.99       0.         0.67598215]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.98009999 0.99       0.97029898]
 [0.98009999 0.98999997 1.         0.9801    ]
 [0.         0.         0.         0.        ]]


## Etapa 7: Avaliar o Agente Treinado

Agora, vamos desligar a exploração (`epsilon = 0`) e ver como o agente se sai em 10 jogos.

In [ ]:
num_jogos_teste = 10
vitorias = 0
frames = [] # Para armazenar os frames da animação

print("Iniciando testes com o agente treinado...")

for jogo in range(num_jogos_teste):
    estado, info = env.reset()
    terminado = False
    truncado = False
    passos = 0
    print(f"--- Jogo de Teste {jogo + 1} ---")

    while not terminado and not truncado:
        # Sempre escolher a MELHOR ação (sem exploração)
        acao = np.argmax(q_table[estado, :])

        novo_estado, recompensa, terminado, truncado, info = env.step(acao)

        # Renderizar e salvar o frame
        frame = env.render()
        print(frame)
        print(f"Passo: {passos+1}, Ação: {acao}")
        time.sleep(0.2) # Pausa para podermos assistir

        estado = novo_estado
        passos += 1

        if recompensa == 1.0:
            print("Vitória! 🏆")
            vitorias += 1
        elif terminado:
            print("Caiu no buraco... 🕳️")

print(f"\nTaxa de vitória em {num_jogos_teste} jogos: {vitorias / num_jogos_teste * 100}%")
env.close()

Iniciando testes com o agente treinado...
--- Jogo de Teste 1 ---
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 1, Ação: 1
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 2, Ação: 1
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 3, Ação: 2
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 4, Ação: 1
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 5, Ação: 2
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 6, Ação: 2
Vitória! 🏆
--- Jogo de Teste 2 ---
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 1, Ação: 1
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 2, Ação: 1
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 3, Ação: 2
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 4, Ação: 1
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 5, Ação: 2
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 6, Ação: 2
Vitória! 🏆
--- Jogo de Teste 3 ---
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 1, Ação: 1
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 2, Ação: 1
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 3, Ação: 2
  (Down)
SFFF
FHFH
FFFH
HFFG

Passo: 4, Ação: 1
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 5, Ação: 2
  (Right)
SFFF
FHFH
FFFH
HFFG

Passo: 6,

## Próximos Passos (Desafios)

1.  **Tente com `is_slippery=True`:** No modo "escorregadio", o agente tem ~33% de chance de ir para uma direção diferente da que ele escolheu. Isso torna o problema muito mais difícil e realista. Será que o Q-Learning ainda funciona?

2.  **Ajuste os Hiperparâmetros:** Tente mudar `alpha`, `gamma` e a `taxa_decaimento_epsilon`. Como isso afeta a velocidade e a qualidade do treinamento?

3.  **Problemas Maiores:** O Q-Learning Tabular funciona bem para 16 estados. Mas e se tivéssemos 16 *milhões* de estados (como no Atari)? A tabela fica grande demais.

É aí que entra o **Deep Q-Network (DQN)** que vimos na aula: substituímos a *tabela* por uma *Rede Neural*.